In [74]:
using JuMP
using HiGHS
using Printf

In [75]:
model = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [76]:
@variable(model, x1s >= 0)  # Crude 1 for Super Gasoline
@variable(model, x2s >= 0)  # Crude 2 for Super Gasoline
@variable(model, x3s >= 0)  # Crude 3 for Super Gasoline
@variable(model, x1r >= 0)  # Crude 1 for Regular Gasoline
@variable(model, x2r >= 0)  # Crude 2 for Regular Gasoline
@variable(model, x3r >= 0)  # Crude 3 for Regular Gasoline
@variable(model, x1d >= 0)  # Crude 1 for Diesel Fuel
@variable(model, x2d >= 0)  # Crude 2 for Diesel Fuel
@variable(model, x3d >= 0)  # Crude 3 for Diesel Fuel

x3d

In [77]:
#total octane and iron
@variable(model, total_octane_s >= 0)
@variable(model, total_octane_r >= 0)
@variable(model, total_octane_d >= 0)
@variable(model, total_iron_s >= 0)
@variable(model, total_iron_r >= 0)
@variable(model, total_iron_d >= 0)

total_iron_d

In [78]:
#objective
@objective(model, Max, 70 * (x1s + x2s + x3s) + 60 * (x1r + x2r + x3r) + 50 * (x1d + x2d + x3d) - 45 * (x1s + x1r + x1d) - 35 * (x2s + x2r + x2d) - 25 * (x3s + x3r + x3d))

25 x1s + 35 x2s + 45 x3s + 15 x1r + 25 x2r + 35 x3r + 5 x1d + 15 x2d + 25 x3d

In [79]:
#total octane constraints
@constraint(model, total_octane_s == 12 * x1s + 6 * x2s + 8 * x3s)
@constraint(model, total_octane_r == 12 * x1r + 6 * x2r + 8 * x3r)
@constraint(model, total_octane_d == 12 * x1d + 6 * x2d + 8 * x3d)

-12 x1d - 6 x2d - 8 x3d + total_octane_d = 0

In [80]:
@constraint(model, total_iron_s == 0.5 * x1s + 2.0 * x2s + 3.0 * x3s)
@constraint(model, total_iron_r == 0.5 * x1r + 2.0 * x2r + 3.0 * x3r)
@constraint(model, total_iron_d == 0.5 * x1d + 2.0 * x2d + 3.0 * x3d)

-0.5 x1d - 2 x2d - 3 x3d + total_iron_d = 0

In [81]:
@constraint(model, total_octane_s >= 10 * (x1s + x2s + x3s))
@constraint(model, total_octane_r >= 8 * (x1r + x2r + x3r))
@constraint(model, total_octane_d >= 6 * (x1d + x2d + x3d))

-6 x1d - 6 x2d - 6 x3d + total_octane_d ≥ 0

In [82]:
@constraint(model, total_iron_s <= 1 * (x1s + x2s + x3s))
@constraint(model, total_iron_r <= 2 * (x1r + x2r + x3r))
@constraint(model, total_iron_d <= 1 * (x1d + x2d + x3d))

-x1d - x2d - x3d + total_iron_d ≤ 0

In [83]:
@constraint(model, x1s + x1r + x1d <= 5000)  # Crude 1 limit
@constraint(model, x2s + x2r + x2d <= 5000)  # Crude 2 limit
@constraint(model, x3s + x3r + x3d <= 5000)  # Crude 3 limit

x3s + x3r + x3d ≤ 5000

In [84]:
@constraint(model, x1s + x2s + x3s + x1r + x2r + x3r + x1d + x2d + x3d <= 14000)

x1s + x2s + x3s + x1r + x2r + x3r + x1d + x2d + x3d ≤ 14000

In [85]:
optimize!(model)
println("Optimal solution found:")
println("Super Gasoline: Crude 1 = ", value(x1s), ", Crude 2 = ", value(x2s), ", Crude 3 = ", value(x3s))
println("Regular Gasoline: Crude 1 = ", value(x1r), ", Crude 2 = ", value(x2r), ", Crude 3 = ", value(x3r))
println("Diesel Fuel: Crude 1 = ", value(x1d), ", Crude 2 = ", value(x2d), ", Crude 3 = ", value(x3d))
println("Optimal profit: \$", objective_value(model))

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-01, 1e+01]
  Cost   [5e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 1e+04]
Presolving model
16 rows, 15 cols, 66 nonzeros  0s
10 rows, 9 cols, 33 nonzeros  0s
9 rows, 9 cols, 31 nonzeros  0s
Presolve : Reductions: rows 9(-7); columns 9(-6); elements 31(-35)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.2499983049e+02 Ph1: 8(28); Du: 9(225) 0s
          6    -3.7500000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  3.7500000000e+05
HiGHS run time      :          0.04
Optimal solution found:
Super Gasoline: Crude 1 = 2000.0, Crude 2 = 0.0, Crude 3 = 500.0
Regular Gasoline: Crude 1 = 3000.0, Crude 2 = 4000.0, Crude 3 = 4500.0
Diesel Fuel: Crude 1 = 0.0, Crude 2

In [86]:
@constraint(model, x1s + x2s + x3s <= 3000)  # Super Gasoline demand
@constraint(model, x1r + x2r + x3r <= 2000)  # Regular Gasoline demand
@constraint(model, x1d + x2d + x3d <= 1000)  # Diesel Fuel demand

x1d + x2d + x3d ≤ 1000

In [87]:
optimize!(model)

# Print the results
println("Optimal solution found:")
println("Super Gasoline: Crude 1 = ", @sprintf("%.2f", value(x1s)), ", Crude 2 = ", @sprintf("%.2f", value(x2s)), ", Crude 3 = ", @sprintf("%.2f", value(x3s)))
println("Regular Gasoline: Crude 1 = ", @sprintf("%.2f", value(x1r)), ", Crude 2 = ", @sprintf("%.2f", value(x2r)), ", Crude 3 = ", @sprintf("%.2f", value(x3r)))
println("Diesel Fuel: Crude 1 = ", @sprintf("%.2f", value(x1d)), ", Crude 2 = ", @sprintf("%.2f", value(x2d)), ", Crude 3 = ", @sprintf("%.2f", value(x3d)))
println("Optimal profit: \$", @sprintf("%.2f", objective_value(model)))

Coefficient ranges:
  Matrix [5e-01, 1e+01]
  Cost   [5e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 1e+04]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.7499605377e+05 Pr: 1(19000) 0s
          7     1.5000000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  1.5000000000e+05
HiGHS run time      :          0.00
Optimal solution found:
Super Gasoline: Crude 1 = 2400.00, Crude 2 = 0.00, Crude 3 = 600.00
Regular Gasoline: Crude 1 = 800.00, Crude 2 = 0.00, Crude 3 = 1200.00
Diesel Fuel: Crude 1 = 800.00, Crude 2 = 0.00, Crude 3 = 200.00
Optimal profit: $150000.00


In [98]:
@constraint(model, x1s + x2s + x3s <= 4500)  # Super Gasoline demand (updated
@constraint(model, x1r + x2r + x3r <= 2000)  # Regular Gasoline demand
@constraint(model, x1d + x2d + x3d <= 1000)  # Diesel Fuel demand

x1d + x2d + x3d ≤ 1000

In [99]:
optimize!(model)

# Print the results
println("Optimal solution found:")
println("Super Gasoline: Crude 1 = ", @sprintf("%.2f", value(x1s)), ", Crude 2 = ", @sprintf("%.2f", value(x2s)), ", Crude 3 = ", @sprintf("%.2f", value(x3s)))
println("Regular Gasoline: Crude 1 = ", @sprintf("%.2f", value(x1r)), ", Crude 2 = ", @sprintf("%.2f", value(x2r)), ", Crude 3 = ", @sprintf("%.2f", value(x3r)))
println("Diesel Fuel: Crude 1 = ", @sprintf("%.2f", value(x1d)), ", Crude 2 = ", @sprintf("%.2f", value(x2d)), ", Crude 3 = ", @sprintf("%.2f", value(x3d)))
println("Optimal profit: \$", @sprintf("%.2f", objective_value(model)))

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       45
Number of nonzeros in Lagrangian Hessian.............:       36

Total number of variables............................:        9
                     variables with only lower bounds:        9
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       15
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       12

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.2499977e+00 1.33e+00 1.12e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [97]:
using JuMP
using Ipopt

# Define the model
model = Model(Ipopt.Optimizer)

# Define the decision variables for the barrels of each crude oil for each product
@variable(model, x1s >= 0)  # Crude 1 for Super Gasoline
@variable(model, x2s >= 0)  # Crude 2 for Super Gasoline
@variable(model, x3s >= 0)  # Crude 3 for Super Gasoline

@variable(model, x1r >= 0)  # Crude 1 for Regular Gasoline
@variable(model, x2r >= 0)  # Crude 2 for Regular Gasoline
@variable(model, x3r >= 0)  # Crude 3 for Regular Gasoline

@variable(model, x1d >= 0)  # Crude 1 for Diesel Fuel
@variable(model, x2d >= 0)  # Crude 2 for Diesel Fuel
@variable(model, x3d >= 0)  # Crude 3 for Diesel Fuel

# Define the objective function (maximize profit)
@objective(model, Max, 70 * (x1s + x2s + x3s) + 60 * (x1r + x2r + x3r) + 50 * (x1d + x2d + x3d) -
                    (45 * (x1s + x1r + x1d) + 35 * (x2s + x2r + x2d) + 25 * (x3s + x3r + x3d)))

# Add the constraints for the octane ratings (nonlinear)
@constraint(model, (12 * x1s + 6 * x2s + 8 * x3s) / (x1s + x2s + x3s) >= 10)
@constraint(model, (12 * x1r + 6 * x2r + 8 * x3r) / (x1r + x2r + x3r) >= 8)
@constraint(model, (12 * x1d + 6 * x2d + 8 * x3d) / (x1d + x2d + x3d) >= 6)

# Add the constraints for the iron content (nonlinear)
@constraint(model, (0.5 * x1s + 2.0 * x2s + 3.0 * x3s) / (x1s + x2s + x3s) <= 1)
@constraint(model, (0.5 * x1r + 2.0 * x2r + 3.0 * x3r) / (x1r + x2r + x3r) <= 2)
@constraint(model, (0.5 * x1d + 2.0 * x2d + 3.0 * x3d) / (x1d + x2d + x3d) <= 1)

# Add the constraints for the customer demands (with updated super gasoline demand)
@constraint(model, x1s + x2s + x3s <= 4500)  # Super Gasoline demand is now 3500 barrels
@constraint(model, x1r + x2r + x3r <= 2000)  # Regular Gasoline demand is 2000 barrels
@constraint(model, x1d + x2d + x3d <= 1000)  # Diesel Fuel demand is 1000 barrels

# Add the constraints for the crude oil supply limits
@constraint(model, x1s + x1r + x1d <= 5000)  # Crude 1 supply limit
@constraint(model, x2s + x2r + x2d <= 5000)  # Crude 2 supply limit
@constraint(model, x3s + x3r + x3d <= 5000)  # Crude 3 supply limit

# Solve the optimization problem
optimize!(model)

# Display the results
println("Super Gasoline: Crude 1 = ", value(x1s), ", Crude 2 = ", value(x2s), ", Crude 3 = ", value(x3s))
println("Regular Gasoline: Crude 1 = ", value(x1r), ", Crude 2 = ", value(x2r), ", Crude 3 = ", value(x3r))
println("Diesel Fuel: Crude 1 = ", value(x1d), ", Crude 2 = ", value(x2d), ", Crude 3 = ", value(x3d))
println("Optimal profit: $", objective_value(model))


This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       36
Number of nonzeros in Lagrangian Hessian.............:       36

Total number of variables............................:        9
                     variables with only lower bounds:        9
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.2499977e+00 1.33e+00 1.78e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///Users/benjamingoff/Desktop/5533/In[97]#51:27\[90mIn[97]:51:27[0;0m]8;;\
println("Diesel Fuel: Crude 1 = ", value(x1d), ", Crude 2 = ", value(x2d), ", Crude 3 = ", value(x3d))
println("Optimal profit: $[48;2;120;70;70m[0;0m", objective_value(model))
[90m#                         └ ── [0;0m[91midentifier or parenthesized expression expected after $ in string[0;0m